# 06. QLoRAによるファインチューニング実践

`05_lora_concept_demo` では、学習済みのLoRAアダプタの効果を確認しました。
このノートブックでは、いよいよ自分たちの手で**QLoRA (Quantized LoRA)** の学習を実行します。

QLoRAは、4bit量子化されたLLMに対してLoRAチューニングを行う技術で、T4 GPUのようなVRAMが限られた環境でもファインチューニングを可能にします。

## この演習のゴール

**少量の追加学習データ（10件）を使って短時間のファインチューニングを行い、モデルが特定の出力形式（JSON）をより遵守するようになることを確認する。**

成功基準は「JSON形式が壊れにくくなること」であり、回答内容の正しさを追求するものではありません。

## 事前準備

Google Colabで実行する場合、メニューの「ランタイム」→「ランタイムのタイプを変更」で、ハードウェアアクセラレータが「T4 GPU」になっていることを確認してください。

In [ ]:
# 編集禁止セル
# セットアップの確認と共通モジュールのインポート
import os
import sys
import torch
import json
from google.colab import drive

if not os.path.isdir('/content/drive'): drive.mount('/content/drive')
repo_path = '/content/llm_lab'
if not os.path.exists(repo_path):
    !git clone https://github.com/akio-kobayashi/llm_lab.git
os.chdir(repo_path)

# 必要なライブラリのインストール
!pip install -q -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu peft trl datasets gradio
if 'src' not in sys.path: sys.path.append(os.path.abspath('src'))

try:
    from src.common import load_llm, generate_text
    from src.lora import create_lora_model, train_lora
    print('共通モジュールのインポートが完了しました。')
except ImportError as e:
    print(f'共通モジュールのインポートに失敗しました: {e}')

## 学習データの準備

LoRAの学習には、「指示（input）」と「模範解答（output）」のペアのデータセットが必要です。
今回は、`data/lora/lora_train_sample.jsonl` をベースに、**あなた自身で10件の学習データを追加**してもらいます。

### 演習: 学習データを追加する

以下の `my_additional_data` リストに、10件の学習データを追加してください。
**ルール:**
- `input` には、モデルへの質問や指示を記述します。
- `output` には、その `input` に対する**理想的な**応答を、**厳密なJSON形式**で記述します。
- JSONのキーは `{"answer": "...", "evidence": "...", "confidence": "..."}` に統一してください。
- `evidence` は、その答えの根拠です。分からなければ `"該当なし"` としてください。
- `confidence` は、答えの自信度を `"high"`, `"medium"`, `"low"` のいずれかで示します。

In [ ]:
# --- ここを編集 --- #

'''
my_additional_data = [
    # 例1: 事実に基づく質問
    {"input": "日本の現在の総理大臣は誰ですか？", "output": '{"answer": "現在の日本の総理大臣は岸田文雄です。", "evidence": "一般的な知識", "confidence": "high"}'},
    
    # 例2: 存在しない情報についての質問
    {"input": "月にはうさぎが住んでいますか？", "output": '{"answer": "いいえ、月にはうさぎは住んでいません。", "evidence": "科学的な事実", "confidence": "high"}'},

    # (1) ここから下に追加してください
    {"input": "", "output": ''},
    {"input": "", "output": ''},
    {"input": "", "output": ''},
    {"input": "", "output": ''},
    {"input": "", "output": ''},
    {"input": "", "output": ''},
    {"input": "", "output": ''},
    {"input": "", "output": ''}
    # (10) ここまで
]
'''
my_additional_data = [
    # 例1: 事実に基づく質問
    {"input": "日本の現在の総理大臣は誰ですか？", "output": '{"answer": "現在の日本の総理大臣は岸田文雄です。", "evidence": "一般的な知識", "confidence": "high"}'},
    # 例2: 存在しない情報についての質問
    {"input": "月にはうさぎが住んでいますか？", "output": '{"answer": "いいえ、月にはうさぎは住んでいません。", "evidence": "科学的な事実", "confidence": "high"}'},
    # (1) 計算問題
    {"input": "12かける5はいくつですか？", "output": '{"answer": "60です。", "evidence": "数学的計算", "confidence": "high"}'},
    # (2) 歴史的事実
    {"input": "江戸幕府を開いたのは誰ですか？", "output": '{"answer": "徳川家康です。", "evidence": "日本史", "confidence": "high"}'},
    # (3) プログラミング/技術
    {"input": "Pythonでリストの長さを取得する関数は何ですか？", "output": '{"answer": "len()関数です。", "evidence": "Python言語仕様", "confidence": "high"}'},
    # (4) 地理/場所
    {"input": "エッフェル塔がある都市はどこですか？", "output": '{"answer": "フランスのパリです。", "evidence": "地理的データ", "confidence": "high"}'},
    # (5) 主観的な質問（正解がないもの）
    {"input": "世界で一番おいしい果物は何ですか？", "output": '{"answer": "個人の好みによるため、客観的な正解はありません。", "evidence": "主観的判断", "confidence": "medium"}'},
    # (6) 前提が誤っている質問（ひっかけ）
    {"input": "ペンギンは空を飛びますか？", "output": '{"answer": "いいえ、ペンギンは鳥類ですが空を飛ぶことはできません。", "evidence": "生物学的特性", "confidence": "high"}'},
    # (7) 科学的原理
    {"input": "水が沸騰するのは摂氏何度ですか？", "output": '{"answer": "標準気圧下では摂氏100度です。", "evidence": "物理化学的特性", "confidence": "high"}'},
    # (8) 翻訳/言語
    {"input": "「Apple」を日本語に訳すと何ですか？", "output": '{"answer": "「リンゴ」です。", "evidence": "英和翻訳", "confidence": "high"}'}
    # (10) ここまで
]
# --- 編集ここまで --- #

# 編集禁止セル
# 元の学習データと追加データを結合して、新しい学習ファイルを作成
base_train_file = 'data/lora/lora_train_sample.jsonl'
new_train_file = 'data/lora/lora_train_extended.jsonl'

with open(new_train_file, 'w', encoding='utf-8') as f_out:
    # 元のデータを書き込む
    with open(base_train_file, 'r', encoding='utf-8') as f_in:
        for line in f_in:
            f_out.write(line)
    # 追加データを書き込む
    for item in my_additional_data:
        if item['input'] and item['output']:
            f_out.write(json.dumps(item, ensure_ascii=False) + '
')

print(f'新しい学習データファイルを {new_train_file} として保存しました。')
# データ件数の確認
!wc -l {new_train_file}

## モデルのロードとLoRAの準備

学習のベースとなる4bit量子化モデルをロードし、`create_lora_model` 関数を使ってLoRAアダプタを追加します。

In [ ]:
# 編集禁止セル
model, tokenizer = None, None
try:
    model, tokenizer = load_llm(use_4bit=True)
    # LoRAモデルの作成
    lora_model = create_lora_model(model)
except Exception as e:
    print(f'モデルのロードまたはLoRAの準備中にエラーが発生しました: {e}')

## 学習前の性能確認

学習を始める前に、現在のモデルがJSON形式の出力をどの程度守れるかを確認しておきましょう。

In [ ]:
# 編集禁止セル
question = "『古都の探偵録』について、作者とあらすじをJSON形式で教えて。"
instruction = "回答は必ず以下のJSON形式で出力してください。\n{ \"author\": \"...\", \"summary\": \"...\" }"
prompt = f"### 指示:\n{question}\n{instruction}\n\n### 応答:\n"

if 'lora_model' in locals() and lora_model:
    print("--- 学習前の回答 ---")
    generated_text = generate_text(lora_model, tokenizer, prompt, max_new_tokens=128)
    answer = generated_text.split("### 応答:")[-
1].strip()
    print(answer)
else:
    print("モデルがロードされていません。")

## QLoRA学習の実行

`train_lora` 関数を呼び出して、QLoRAの学習を開始します。
データ量とエポック数を絞っているため、**学習は10〜15分程度**で完了します。

In [ ]:
# 編集禁止セル
# LoRAアダプタの保存先をGoogle Drive上に設定
DRIVE_DIR = '/content/drive/MyDrive/llm_lab_outputs'
output_dir = os.path.join(DRIVE_DIR, 'my_lora_adapter')
os.makedirs(output_dir, exist_ok=True)

if 'lora_model' in locals() and lora_model:
    trainer = train_lora(
        model=lora_model,
        tokenizer=tokenizer,
        train_dataset_path=new_train_file,
        output_dir=output_dir,
        max_steps=20, # 学習ステップ数（エポック数ではなくステップ数で指定）
        per_device_train_batch_size=1, # バッチサイズ
        gradient_accumulation_steps=8, # 勾配蓄積数
        learning_rate=2e-4, # 学習率
        max_seq_length=512 # 扱うトークンの最大長
    )
else:
    print("モデルが準備できていないため、学習をスキップします。")

## 学習後の性能確認

学習が完了したら、学習前と**全く同じ質問**をして、出力がどう変わったかを確認しましょう。
うまく学習できていれば、指示されたJSON形式をより忠実に守るようになっているはずです。

In [ ]:
# 編集禁止セル
if 'lora_model' in locals() and lora_model:
    print("--- 学習後の回答 ---")
    # 学習後のモデルで再度テキスト生成
    generated_text = generate_text(lora_model, tokenizer, prompt, max_new_tokens=128)
    answer = generated_text.split("### 応答:")[-
1].strip()
    print(answer)
else:
    print("モデルがロードされていません。")

## まとめ

このノートブックでは、QLoRAによるファインチューニングを実践しました。

- **データが重要**: ファインチューニングの性能は、学習データの質と量に大きく依存します。今回は少量でしたが、データ件数を増やし、多様な例を与えることで、より頑健なモデルを作ることができます。
- **短時間での適応**: QLoRAにより、限られた計算資源でも、特定のタスクにモデルを効率的に適応させることができました。
- **アダプタの保存**: 学習したアダプタは `my_lora_adapter/final_adapter` ディレクトリに保存されています。このアダプタをベースモデルに適用することで、いつでも学習後の性能を再現できます。

最後の `07_integrate_gradio.ipynb` では、この演習で学んだRAGとLoRAの技術を組み合わせて、一つのアプリケーションとして完成させます。

### メモリ解放

In [ ]:
# 編集禁止セル
import gc
if 'model' in locals() and model is not None: del model
if 'lora_model' in locals() and lora_model is not None: del lora_model
if 'tokenizer' in locals() and tokenizer is not None: del tokenizer
if 'trainer' in locals() and trainer is not None: del trainer
model, lora_model, tokenizer, trainer = None, None, None, None
gc.collect()
torch.cuda.empty_cache()
print("モデルを解放し、GPUキャッシュをクリアしました。")